# 

## nazwa plikow po blascie
query_db_bl

In [1]:
import os
import pandas as pd
from bioinfokit.analys import Fasta
from Bio import SeqIO
# helpers function

def get_fasta(filename):
    n = 0
    sequence = ""
    te_name = ""
    with open(filename) as f:
        for line in f.readlines():
            if line.startswith('>'):
                n = n + 1
                if not n == 1:
                    first_n = False
                    yield te_name, sequence
                te_name = line[1:].strip()
                sequence = ""
            else:
                if te_name:
                    sequence = sequence + line.strip()
    yield te_name, sequence

def get_len_fasta(filename):
    counter = {}
    with open(filename) as f:
        te_name = ""
        for line in f.readlines():
            if line.startswith('>'):
                te_name = line[1:].strip()
                counter[te_name] = 0
            else:
                if te_name:
                    counter[te_name] = counter[te_name] + len(line.strip()) - line.count("~")
    return counter

def get_from_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len - te_len * dif_percent / 100)

def get_to_te(fasta, te_name):
    te_len = get_len_fasta(fasta)[te_name]
    return int(te_len + te_len * dif_percent / 100)

def read_fasta(filename):
    return pd.read_csv(filename, sep="\t", header=None).rename(headers_dict, axis=1)

def save_fasta(df, filename):
    df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)








In [10]:
ont = ["K10B", "K10f", "LK5_1_3"]
ont = ["K10B", "K10f"]
ont = ["ccc"]
ont = [
    "K10B",
    "K10f",
    "LK5_1_3"
]
genome_path = "genome/DCARv3.4.fa"
genome_path = "genome/genomic2.fna"
te_path = "te/Alex1_3.fas"
dif_percent = 30

te_name = 'Alex1'
te_len = get_len_fasta(te_path)[te_name]
from_te = get_from_te(te_path, te_name)
to_te = get_to_te(te_path, te_name)



mask_genome(te_name, te_path, genome_path, True)
correct_first_part(False, True, True)
#oxi = correct_second_part()
#save_report(oxi)
#generate_second_report("report_Alex1__K10B_K10f", "doro")





Building a new DB, current time: 05/21/2024 09:30:28
New DB name:   /home/kiki/data2/genome/genome
New DB title:  genome/genomic2.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data2/genome/genome
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 4.54529 seconds.




okno
makeblastdb -in ont/K10B.fasta -dbtype nucl -out ont/K10B


Building a new DB, current time: 05/21/2024 09:30:33
New DB name:   /home/kiki/data2/ont/K10B
New DB title:  ont/K10B.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data2/ont/K10B
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 6939343 sequences in 234.7 seconds.


makeblastdb -in ont/K10f.fasta -dbtype nucl -out ont/K10f


Building a new DB, current time: 05/21/2024 09:34:29
New DB name:   /home/kiki/data2/ont/K10f
New DB title:  ont/K10f.fasta
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data2/ont/K10f
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 4775182 sequences in 160.619 seconds.


makeblastdb -in ont/LK5_1_3.fasta -dbtype nucl -out ont/LK5_1_3


Building a new DB, current time: 05/21/2024 09:37:11
New DB name:   /home/kiki/data2/ont/LK5_1_3
New DB title:  ont/LK5_1

blastn -num_threads 20 -outfmt 6 -query te/Alex1_3.fas -db ont/K10f -out blast/TE_K10f_Alex1.bl -dust no -perc_identity 0.9


blastn -num_threads 20 -outfmt 6 -query te/Alex1_3.fas -db ont/LK5_1_3 -out blast/TE_LK5_1_3_Alex1.bl -dust no -perc_identity 0.9


first part
second part
1 part
first/p_normal_K10B_Alex1.fasta
first/m_normal_K10B_Alex1.fasta
first part
second part
1 part
first/p_normal_K10f_Alex1.fasta
first/m_normal_K10f_Alex1.fasta
first part
second part
1 part
first/p_normal_LK5_1_3_Alex1.fasta
first/m_normal_LK5_1_3_Alex1.fasta


In [11]:
headers = ['qseqid', 'sseqid',  'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'expect value', 'bitscore']
headers_dict = {i[0]:i[1] for i in enumerate(headers)}

In [12]:
import random
def rand_split(sequence, splits):
    for splitLen in splits:
        if splitLen > len(sequence):
            break
        yield sequence[:splitLen]
        sequence = sequence[splitLen::]

def rand_gen(int_list):
    while True:
        yield int(random.choice(int_list))

def cut_genome(genome_path, ont_path, coverage, filename):
    possibility_len = list(get_len_fasta(ont_path).values())
    n = 0
    for i in range(coverage):
        print(i)
        for name, genome_sequence in get_fasta(genome_path):
            print(name)
            with open(filename, 'a') as file:
                for seq in rand_split(genome_sequence, rand_gen(possibility_len)):
                    n = n + 1
                    file.write(f">{name}_{i}_{n}\n{seq}\n")
#cut_genome(genome_path, "ont/K10f.fasta", 20, 'cutted_genome.fasta')

In [7]:
def mask_genome(te_name, te_path, genome_path, make_db=True):
    os.system(f"mkdir -p masked_genome")
    if make_db:
        os.system(f"makeblastdb -in {genome_path} -dbtype nucl -out genome/genome")
    os.system(f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db genome/genome -out masked_genome/{te_name}_genome.bl -perc_identity 0.9")
    te_genome_bl = read_fasta(f"masked_genome/{te_name}_genome.bl")
    te_genome_bl = te_genome_bl[te_genome_bl.qseqid == te_name]
    te_genome_bl["start"] = te_genome_bl['sstart'].where(te_genome_bl['sstart'] <= te_genome_bl["send"], other=te_genome_bl['send'])
    te_genome_bl["stop"] = te_genome_bl['sstart'].where(te_genome_bl['sstart'] > te_genome_bl["send"], other=te_genome_bl['send'])
    # todo
    save_fasta(te_genome_bl[["sseqid", "start", "stop"]], f"masked_genome/to_mask_genome_{te_name}.txt")
    plus_te_genome_bl = te_genome_bl[te_genome_bl.sstart < te_genome_bl.send]
    minus_te_genome_bl = te_genome_bl[te_genome_bl.sstart > te_genome_bl.send]

    merge_headers = ["sseqid", "sstart", "send"]
    save_fasta(plus_te_genome_bl[merge_headers].sort_values(by=merge_headers), f"masked_genome/plus_to_mask_genome_{te_name}.bed")
    merge_headers = ["sseqid", "send", "sstart"]
    save_fasta(minus_te_genome_bl[merge_headers].sort_values(by=merge_headers), f"masked_genome/minus_to_mask_genome_{te_name}.bed")

    os.system(f"bedtools merge -i masked_genome/plus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/plus_to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools merge -i masked_genome/minus_to_mask_genome_{te_name}.bed -d 200 > masked_genome/minus_to_mask_genome_{te_name}.bed.merged")
    minus_merged =  pd.read_csv(f"masked_genome/minus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    plus_merged =  pd.read_csv(f"masked_genome/plus_to_mask_genome_{te_name}.bed.merged", sep="\t", header=None)
    merged_genome = pd.concat(
        [
            minus_merged[
                (minus_merged[2] - minus_merged[1] > from_te)
                & (minus_merged[2] - minus_merged[1] < to_te)
            ],
            plus_merged[
                (plus_merged[2] - plus_merged[1] > from_te)
                & (plus_merged[2] - plus_merged[1] < to_te)
            ],
        ]
    )
    merged_genome["length"] = merged_genome[2] - merged_genome[1]
    save_fasta(merged_genome, f"masked_genome/to_mask_genome_{te_name}.bed.merged")
    os.system(f"bedtools maskfasta -fi {genome_path} -bed masked_genome/to_mask_genome_{te_name}.bed.merged -fo genome/{te_name}_masked_genome")
    return merged_genome
masked_genome = mask_genome(te_name, te_path, genome_path, True)



Building a new DB, current time: 05/21/2024 09:29:57
New DB name:   /home/kiki/data2/genome/genome
New DB title:  genome/genomic2.fna
Sequence type: Nucleotide
Deleted existing Nucleotide BLAST database named /home/kiki/data2/genome/genome
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 9 sequences in 4.27525 seconds.




In [8]:
masked_genome

,0,1,2,length
1,chromosome2,30539632,30544357,4725
4,chromosome3,27717880,27722606,4726
6,chromosome5,8967214,8971939,4725
9,chromosome9,9220466,9226148,5682
10,chromosome9,28529339,28534568,5229
0,chromosome1,18795334,18799058,3724
2,chromosome3,41159456,41164182,4726
5,chromosome4,35827649,35832374,4725


In [9]:
# first part
def  create_subsequent_fasta(fasta, bed_df, file_base):
    # create subsequents with left and right flanks
    # site p or m
    num = 4000

    print(fasta)
    records = SeqIO.parse(fasta, 'fasta')
    record_dict = SeqIO.to_dict(records)
    for bed_rec in  bed_df.itertuples():
        seq_id = bed_rec.sseqid
        record = record_dict[seq_id]
        
        te_start = bed_rec.sstart
        te_end = bed_rec.send

        left_start = te_start - num if te_start - num > 0 else 0
        left_end = te_start

        right_start = te_end
        right_end = te_end + num if te_end + num < len(record) else len(record)

        left_subseq_record = record[left_start:left_end]  # For the full record (with header)
        right_subseq_record = record[right_start:right_end]

        with open(f"first/left_{file_base}", 'a') as file:
                file.write(f">{record.id}\n{left_subseq_record.seq}\n")
        with open(f"first/right_{file_base}", 'a') as file:
                file.write(f">{record.id}\n{right_subseq_record.seq}\n")

def correct_first_part(fq=True, db=True, do_blast=True):
    os.system("mkdir -p blast")
    os.system("mkdir -p first")
    if fq:
        for file in ont:
            os.system(f"seqtk seq -a ont/{file}.fastq.gz > ont/{file}.fasta")
    if db:
        print('okno')
        for file in ont:
            print(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
            os.system(f"makeblastdb -in ont/{file}.fasta -dbtype nucl -out ont/{file}")
    if do_blast:
        print('okno2')
        for file in ont:
            print(
                f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db ont/{file} " \
                f"-out blast/TE_{file}_{te_name}.bl -dust no -perc_identity 0.9"
            )
            os.system(
                f"blastn -num_threads 20 -outfmt 6 -query {te_path} -db ont/{file} " \
                f"-out blast/TE_{file}_{te_name}.bl -dust no -perc_identity 0.9"
            )

    for o in ont:
        te_ont_bl = read_fasta(f"blast/TE_{o}_{te_name}.bl")
        te_ont_bl = te_ont_bl[te_ont_bl.qseqid.str.contains(te_name)]
        plus_te_ont_bl = te_ont_bl[te_ont_bl.sstart < te_ont_bl.send]
        minus_te_ont_bl = te_ont_bl[te_ont_bl.sstart > te_ont_bl.send]
    
    
        plus_te_ont_bed = plus_te_ont_bl[['sseqid', 'sstart', 'send', 'qseqid']].sort_values(by=['sseqid', 'sstart', 'send'])
        minus_te_ont_bed = minus_te_ont_bl[['sseqid', 'send', 'sstart', 'qseqid']].sort_values(by=['sseqid', 'send', 'sstart'])
    
        save_fasta(minus_te_ont_bed, f'first/m_raw_first_blast_{o}_{te_name}.bed')
        save_fasta(plus_te_ont_bed, f'first/p_raw_first_blast_{o}_{te_name}.bed')

        os.system(f"bedtools merge -i first/p_raw_first_blast_{o}_{te_name}.bed -d 1500 -c 4 -o collapse  > first/p_first_blast_{o}_{te_name}.bed.merged")
        os.system(f"bedtools merge -i first/m_raw_first_blast_{o}_{te_name}.bed -d 1500 -c 4 -o collapse  > first/m_first_blast_{o}_{te_name}.bed.merged")

        print("first part")
    
        plus_merged = pd.read_csv(
            f"first/p_first_blast_{o}_{te_name}.bed.merged", sep="\t", header=None).rename(
            {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
        )
        minus_merged = pd.read_csv(
            f"first/m_first_blast_{o}_{te_name}.bed.merged", sep="\t", header=None).rename(
            {0:'sseqid', 1:'sstart', 2:'send', 3:'qseqid'}, axis=1
        )
    
        plus_filtered_bed = plus_merged[(plus_merged.send - plus_merged.sstart) > from_te]
        minus_filtered_bed = minus_merged[(minus_merged.send - minus_merged.sstart) > from_te]

        # todo in this place

        save_fasta(pd.DataFrame({'id': plus_filtered_bed['sseqid'].unique()}), f'first/p_seq_list_{o}_{te_name}.txt')
        save_fasta(pd.DataFrame({'id': minus_filtered_bed['sseqid'].unique()}), f'first/m_seq_list_{o}_{te_name}.txt')
        #df.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=False)
        print("second part")
    
        #long
        # it's filtered only our fasta
        os.system(f"seqtk subseq ont/{o}.fasta first/p_seq_list_{o}_{te_name}.txt > first/p_normal_{o}_{te_name}.fasta")
        os.system(f"seqtk subseq ont/{o}.fasta first/m_seq_list_{o}_{te_name}.txt > first/m_normal_{o}_{te_name}.fasta")
        print("1 part")

        create_subsequent_fasta(f"first/p_normal_{o}_{te_name}.fasta", plus_filtered_bed,  f"p_{o}_{te_name}.fasta")
        create_subsequent_fasta(f"first/m_normal_{o}_{te_name}.fasta", minus_filtered_bed,  f"m_{o}_{te_name}.fasta")

# Second part of analysis

In [108]:


"""
x(start_df, end_df, strain, ont_file) #cale 2 df
xx(start_record, end_df, strain) # jesli znajdzie dopasowuje prawo
xxx(start_record, end_record) # True False

"""


#second part
class Blast:
    @staticmethod
    def make_db(fasta_file, out):
        os.system(f"makeblastdb -in {fasta_file} -dbtype nucl -out {out}")

    @staticmethod
    def run(query, db, out):
        os.system(f"blastn -num_threads 20 -outfmt 6 -query {query} -db {db} -out {out} -perc_identity 0.9")
        return pd.read_csv(out, sep="\t", header=None).rename(headers_dict, axis=1)

class SecondService:
    def __init__(self, filename, te_name):
        self.filename = filename
        self.te_name = te_name

        self.plus_te_start_records = {}
        self.plus_te_end_records = {}
        self.minus_te_start_records = {}
        self.minus_te_end_records = {}

    @property
    def used_ids(self):
        used_records = set()
        for _, records in self.plus_te_start_records.items():
            for record in records:
                used_records.add(record.qseqid)
        for _, records in self.minus_te_start_records.items():
            for record in records:
                used_records.add(record.qseqid)
        for _, records in self.plus_te_end_records.items():
            for record in records:
                used_records.add(record.qseqid)
        for _, records in self.minus_te_end_records.items():
            for record in records:
                used_records.add(record.qseqid)
        return used_records
    
    def add_start(self, record, ont_file, is_plus_strain):
        if is_plus_strain:
            if self.plus_te_start_records.get(ont_file) is None:
                self.plus_te_start_records[ont_file] = [record]
            else:
                self.plus_te_start_records[ont_file].append(record)
        else:
            if self.minus_te_start_records.get(ont_file) is None:
                self.minus_te_start_records[ont_file] = [record]
            else:
                self.minus_te_start_records[ont_file].append(record)

    def add_end(self, record, ont_file, is_plus_strain):
        if is_plus_strain:
            if self.plus_te_end_records.get(ont_file) is None:
                self.plus_te_end_records[ont_file] = [record]
            else:
                self.plus_te_end_records[ont_file].append(record)
        else:
            if self.minus_te_end_records.get(ont_file) is None:
                self.minus_te_end_records[ont_file] = [record]
            else:
                self.minus_te_end_records[ont_file].append(record)
        
    
    def get_start_field(self, is_plus_strain):
        return "send" if is_plus_strain else "sstart"

    def get_end_field(self, is_plus_strain):
        return "sstart" if is_plus_strain else "send"

    def if_match(self, start_record, end_record, is_plus_strain):
        start = getattr(start_record, self.get_start_field(is_plus_strain))
        end = getattr(end_record, self.get_end_field(is_plus_strain))
        return (end - start < to_te and end - start > from_te) or abs(end - start) < 200

    def match_rec_df(self, start_record, end_df, is_plus_strain):
        chromosome = start_record.sseqid
        r_df =  end_df[(end_df.sseqid == chromosome)]
        for right_row in r_df.itertuples():
            if self.if_match(start_record, right_row, is_plus_strain):
                return right_row
        return None

    '''
    def match_not_ideal_pairs(self, left_df, right_df, is_plus_strain):
        start_df = left_df.sort_values(by=['sseqid', 'length', self.get_start_field(is_plus_strain)])
        end_df = right_df.sort_values(by=['sseqid', 'length', self.get_end_field(is_plus_strain)])
        if not len(start_df) or not len(end_df):
            return False
            if not len(start_df):
                if is_plus_strain:
                    self.plus_te_end_records.append(end_df.iloc[0])
                else:
                    self.minus_te_end_records.append(end_df.iloc[0])
                return True
            elif not len(end_df):
                if is_plus_strain:
                    self.plus_te_start_records.append(start_df.iloc[0])
                else:
                    self.minus_te_start_records.append(start_df.iloc[0])
                return True
    
            for chromosome in start_df.sseqid.unique():
                l_df =  start_df[(start_df.sseqid == chromosome)]
                r_df =  end_df[(end_df.sseqid == chromosome)]
                
        return False
    '''
    
    def match_ideal_pairs(self, left_df, right_df, ont_file, is_plus_strain):
        # add left and right if find
        # iterate chromosome
        # only one id
        start_df = left_df.sort_values(by=['sseqid', 'length', self.get_start_field(is_plus_strain)])
        end_df = right_df.sort_values(by=['sseqid', 'length', self.get_end_field(is_plus_strain)])
        if not len(start_df) or not len(end_df):
            return False
        for chromosome in start_df.sseqid.unique():
            l_df =  start_df[(start_df.sseqid == chromosome)]
            r_df =  end_df[(end_df.sseqid == chromosome)]
            for start_row in l_df.itertuples():
                matched_right_record = self.match_rec_df(start_row, r_df, is_plus_strain)
                if  matched_right_record is not None:
                    if is_plus_strain:
                        self.add_start(start_row, ont_file, is_plus_strain)
                        self.add_end(matched_right_record, ont_file, is_plus_strain)
                    return True
                
        return False
    
    def save_to_file(self, record, is_start, is_plus_strain, ont_file):
        if record is not None:
            r = {
                "chromosome":record.sseqid,
                "strain":"plus" if is_plus_strain else "minus",
                "start":getattr(record, self.get_start_field(is_plus_strain)) if is_start else None,
                "stop":getattr(record, self.get_end_field(is_plus_strain)) if not is_start else None,
                "ont_id":f"{ont_file}:{record.qseqid}",
            }
            with open(self.filename, "a") as f_out:
                f_out.write(
                    f"{r['chromosome']}\t{r['strain']}\t{r['start']}\t{r['stop']}\t{r['ont_id']}\n"
                )
    
    def match_dfs(self, left_df, right_df, ont_file, is_plus_strain):
        # iterate ont_ids
        all_ids = set(left_df.qseqid.unique()).union(set(right_df.qseqid.unique()))
        # msstch 1
        for ont_id in all_ids:
            left = left_df[left_df.qseqid == ont_id]
            right = right_df[right_df.qseqid == ont_id]
            self.match_ideal_pairs(left, right, ont_file, is_plus_strain)

        
        # todo - now
        not_used_start_records = left_df[~left_df.qseqid.isin(self.used_ids)]
        not_used_end_records = right_df[~right_df.qseqid.isin(self.used_ids)]
        used_start_records = left_df[left_df.qseqid.isin(self.used_ids)]
        used_end_records = right_df[right_df.qseqid.isin(self.used_ids)]
        for not_used_start_record in not_used_start_records.itertuples():
            for used_start_record in used_start_records.itertuples():
                start1 = getattr(not_used_start_record, self.get_start_field(is_plus_strain))
                start2 = getattr(used_start_record, self.get_start_field(is_plus_strain))
                if abs(start1 - start2) < 300:
                    print('konik')
        
        
    
    # main function
    def xxx(self, left_blast, right_blast, ont_file):
        left_blast =  left_blast[left_blast.length > 600]
        right_blast =  right_blast[right_blast.length > 600]
    
        plus_left = left_blast[left_blast.sstart <= left_blast.send]
        plus_left["strain"] = "+"
        plus_right =  right_blast[right_blast.sstart <= right_blast.send]
        plus_right["strain"] = "+"
        
        minus_left = left_blast[left_blast.sstart > left_blast.send]
        minus_left["strain"] = "-"
        minus_right = right_blast[right_blast.sstart > right_blast.send]
        minus_right["strain"] = "-"

        self.match_dfs(plus_left, plus_right, ont_file, True)
        self.match_dfs(minus_right, minus_left, ont_file, False)

    def ont_to_genome_blast(self, fasta_filename, blast_filename):
        return Blast.run(
            fasta_filename,
            f"genome/masked_genome",
            blast_filename,
        ).sort_values(['qseqid', 'bitscore'], ascending=False).groupby(['qseqid'], as_index=False).first()

    def blast(self, o, blast_db=True):
        left_m = self.ont_to_genome_blast(
            f"first/left_m_{o}_{self.te_name}.fasta",
            f"second/left_m_{o}_{self.te_name}_genome.bl",
        )
        right_m = self.ont_to_genome_blast(
            f"first/right_m_{o}_{self.te_name}.fasta",
            f"second/right_m_{o}_{self.te_name}_genome.bl",
        )
        left_p = self.ont_to_genome_blast(
            f"first/left_p_{o}_{self.te_name}.fasta",
            f"second/left_p_{o}_{self.te_name}_genome.bl",
        )
        right_p = self.ont_to_genome_blast(
            f"first/right_p_{o}_{self.te_name}.fasta",
            f"second/right_p_{o}_{self.te_name}_genome.bl",
        )
        return [
            [left_m, right_m], [left_p, right_p]
        ]
    
    def run(self, blast_db=True):

        raw_report_header = ["chromosome", "strain", "start", "stop", "ont_id"]
        header_to_save = '\t'.join(raw_report_header)
        with open(self.filename, "w") as f_out:
            f_out.write(
                f"{header_to_save}\n"
            )
    
        report = {}
        os.system("mkdir -p final")
        os.system("mkdir -p second")
        pd.options.mode.copy_on_write = True
        if blast_db:
            Blast.make_db(genome_path, "genome/masked_genome")
        for o in ont:
            for left_blast, right_blast in self.blast(o, blast_db):
                self.xxx(left_blast, right_blast, o)
    
        for ont_file, start_records in self.plus_te_start_records.items():
            for start_record in start_records:
                self.save_to_file(start_record, True, True, ont_file)
        for ont_file, end_records in self.plus_te_end_records.items():
            for end_record in end_records:
                self.save_to_file(end_record, False, True, ont_file)
            
        for ont_file, start_records in self.minus_te_start_records.items():
            for start_record in start_records:
                self.save_to_file(start_record, True, False, ont_file)
        for ont_file, end_records in self.minus_te_end_records.items():
            for end_record in end_records:
                self.save_to_file(end_record, False, False, ont_file)
        #report.to_csv(self.filename, sep='\t', index=False, encoding='utf-8', header=True)
        return pd.read_csv(self.filename, sep="\t")
                

In [109]:
filename = "zizwa"
service = SecondService(filename, te_name)
raw_report = service.run(False)

konik


konik
konik
konik
konik
konik
konik
konik
konik
konik
konik
konik
konik


In [36]:
len(raw_report)

90

In [37]:
raw_report.head()

,chromosome,strain,start,stop,ont_id
0,chromosome9,plus,28530068.0,NaN,K10B:e9b31c79-83c5-4cbf-a699-684507b19baa
1,chromosome9,plus,28529338.0,NaN,K10B:10e616cd-c2ca-42e2-a391-8078ac0d8956
2,chromosome9,plus,9220466.0,NaN,K10B:070100af-93c2-4c1c-b426-0e6852ff5740
3,chromosome9,plus,28529396.0,NaN,K10B:c20f1362-fbcb-4230-8a32-73659bc06318
4,chromosome5,plus,41745603.0,NaN,K10B:d69243ea-29eb-45f5-9a67-cb1adf455e12


In [64]:
import numpy as np


def get_first_group(df, field):
    first_row = df.iloc[0]
    field_value = first_row[field]
    #group = df[(df[field] > field_value - 1700) & (df[field] < field_value + 1700)]
    #new_df = df[(df[field] <= field_value - 1700) | (df[field] >= field_value + 1700)]
    group = df[(df[field] > field_value - 1000) & (df[field] < field_value + 1000)]
    new_df = df[(df[field] <= field_value - 1000) | (df[field] >= field_value + 1000)]
    return new_df, group

def get_groups_dict(groups, field):
    group_dict = {}
    for group in groups:
        first_row = group.iloc[0]
        group_dict[first_row.chromosome, int(group[field].mode().median())] = group
    return group_dict

def get_groups(records, field):
    all_records = records[raw_report[field].notnull()]
    groups = []
    for chromosome in all_records.chromosome.unique():
        remaining = all_records[all_records.chromosome == chromosome]
        while len(remaining):
            g = get_first_group(remaining, field)
            remaining = g[0]
            groups.append(g[1])
    return groups



def if_match(start_chromosome, end_chromosome, start, end):
    if start_chromosome == end_chromosome:
        if (end - start > from_te and end - start < to_te) or (abs(end - start) < 200):
            return True
    return False

def create_record(chromosome, start, end, start_records, end_records):
    start_ids = set([ont_id for ont_id in start_records.ont_id])
    end_ids = set([ont_id for ont_id in end_records.ont_id])
    if start_ids.intersection(end_ids):
        pkt = 1
    elif set([x.split(":")[0] for x in start_ids]).intersection([x.split(":")[0] for x in end_ids]):
        pkt = 2
    elif start_ids and start_ids:
        pkt = 3
    else:
        pkt = 4
    return  {
        "pkt":pkt,
        "chromosome": chromosome,
        "start":start,
        "stop":end,
        "length":end-start,
        "left_n":len(start_records),
        "left_rec":",".join([ont_id for ont_id in start_records.ont_id]),
        "right_n":len(end_records),
        "right_rec":",".join([ont_id for ont_id in end_records.ont_id]),
        }


def match(start_dict, end_dict):
    report_list = []
    start_remaining = start_dict.copy()
    end_remaining = end_dict.copy()
    for (start_chromosome, start), start_records in start_dict.items():
        end_dict = end_remaining.copy()
        for (end_chromosome, end), end_records in end_dict.items():
            if if_match(start_chromosome, end_chromosome, start, end):
                report_list.append(
                    create_record(
                        start_chromosome,
                        start,
                        end,
                        start_records,
                        end_records
                    )
                )
                del start_remaining[start_chromosome, start]
                del end_remaining[end_chromosome, end]
                continue
        #todo if not match?
    return report_list

def save_report(report_list, filename):
    report_header = ["pkt", "chromosome", "start", "stop", "length",  "left_n", "left_rec", "right_n", "right_rec"]
    header_to_save = '\t'.join(report_header)
    with open(filename, "w") as f_out:
        f_out.write(
            f"{header_to_save}\n"
        )
    for r in report_list:
        with open(filename2, "a") as f_out:
            f_out.write(
                f"{r['pkt']}\t{r['chromosome']}\t{r['start']}\t{r['stop']}\t{r['length']}\t"
                f"{r['left_n']}\t{r['left_rec']}\t{r['right_n']}\t{r['right_rec']}\n"
            )

def save_first_point_report(filename):
    start_groups = get_groups(raw_report, "start")
    end_groups = get_groups(raw_report, "stop")

    start_dict = get_groups_dict(start_groups, "start")
    end_dict = get_groups_dict(end_groups, "stop")

    report_list = match(start_dict, end_dict)

    
    save_report(report_list, filename)

    report = pd.read_csv(filename, sep="\t").sort_values(by=['chromosome', 'pkt', 'start'])
    report.to_csv(filename, sep='\t', index=False, encoding='utf-8', header=True)
    return report

filename2 = "moli2"
first_report = save_first_point_report(filename2)

21


In [51]:
first_report

,pkt,chromosome,start,stop,length,left_n,left_rec,right_n,right_rec
19,1,chromosome1,1895035,1895057,22,3,"LK5_1_3:ad199210-fea3-4427-b40a-288d4cb970a8,L...",3,"LK5_1_3:ad199210-fea3-4427-b40a-288d4cb970a8,L..."
20,1,chromosome1,18795335,18799060,3725,2,"LK5_1_3:f9d75fb2-63a6-4a2d-b898-71b4c2099e3d,L...",2,"LK5_1_3:f9d75fb2-63a6-4a2d-b898-71b4c2099e3d,L..."
7,1,chromosome2,24991939,24991944,5,1,K10B:aa99dd0d-ba11-4f47-a79e-5dfedc9da5e2,1,K10B:aa99dd0d-ba11-4f47-a79e-5dfedc9da5e2
6,1,chromosome2,30539628,30544358,4730,1,K10B:b647706c-82df-4164-9168-4b200501ca6d,1,K10B:b647706c-82df-4164-9168-4b200501ca6d
8,1,chromosome3,27718180,27722607,4427,1,K10B:6c07ed7c-c720-49ac-9e4b-b83309306bae,1,K10B:6c07ed7c-c720-49ac-9e4b-b83309306bae
9,1,chromosome3,35158725,35158723,-2,1,K10B:cf60c0dc-a60a-4a67-acb7-28845f6eeb4c,1,K10B:cf60c0dc-a60a-4a67-acb7-28845f6eeb4c
10,1,chromosome3,41159456,41164176,4720,3,"K10B:7e4c3981-29d5-40a6-aad0-59e1e9322d19,K10B...",3,"K10B:7e4c3981-29d5-40a6-aad0-59e1e9322d19,K10B..."
16,1,chromosome4,4759534,4759532,-2,1,K10B:941656fa-26ed-4578-8aab-59824b70f133,1,K10B:941656fa-26ed-4578-8aab-59824b70f133
17,1,chromosome4,35867416,35867451,35,1,K10B:a7a007a4-c486-4df5-854d-69f74d61d3d1,1,K10B:a7a007a4-c486-4df5-854d-69f74d61d3d1
5,1,chromosome5,8967214,8971940,4726,5,"LK5_1_3:02a6bc39-ecc1-41f0-b5ca-c26c015ff4b1,L...",5,"LK5_1_3:02a6bc39-ecc1-41f0-b5ca-c26c015ff4b1,L..."


In [553]:
masked_genome


,0,1,2,length
1,chromosome2,30539632,30544357,4725
4,chromosome3,27717880,27722606,4726
6,chromosome5,8967214,8971939,4725
9,chromosome9,9220466,9226148,5682
10,chromosome9,28529339,28534568,5229
0,chromosome1,18795334,18799058,3724
2,chromosome3,41159456,41164182,4726
5,chromosome4,35827649,35832374,4725


In [65]:
def _is_ref(df, chromosome, start, stop):
    return (
        (df.chromosome == chromosome)
        & (df.start < start + 500) & (df.start > start - 500)
        & (df.stop < stop + 500) & (df.stop > stop - 500)
    )

def is_ref(df, reference):
    is_reference = None
    for rec in  masked_genome.itertuples():
        chromosome = rec[1]
        start = rec[2]
        stop = rec[3]
        part_is_reference = _is_ref(df, chromosome, start, stop)
        is_reference = is_reference | part_is_reference if is_reference is not None else part_is_reference
    return is_reference

def get_rec(x, o):
    records = []
    for r in x.split(","):
        rec = r.split(":")
        o_file = rec[0]
        o_id = rec[1]
        if o_file == o:
            records.append(o_id)
    return ",".join(records)

def get_num(x, o):
    records = []
    for r in x.split(","):
        rec = r.split(":")
        o_file = rec[0]
        o_id = rec[1]
        if o_file == o:
            records.append(o_id)
    return len(records)

def generate_second_report(filename_in, filename_out, reference):
    report_header = ["pkt", "chromosome", "start", "stop", "length", "is_ref"]
    df =  pd.read_csv(filename2, sep="\t")
    for o in ont:
        df[f"left_{o}_num"] = df.left_rec.apply(
            get_num,
            args = [o]
        )
        df[f"left_{o}_rec"] = df.left_rec.apply(
            get_rec,
            args = [o]
        )
        report_header.append(f"left_{o}_num")
        report_header.append(f"left_{o}_rec")

        df[f"right_{o}_num"] = df.right_rec.apply(
            get_num,
            args = [o]
        )
        df[f"right_{o}_rec"] = df.right_rec.apply(
            get_rec,
            args = [o]
        )
        report_header.append(f"right_{o}_num")
        report_header.append(f"right_{o}_rec")

    is_reff = is_ref(df, masked_genome)
    df['is_ref'] = "nonref"
    df.loc[(df.length > 600), 'is_ref'] = "unknown"    
    df.loc[is_reff, 'is_ref'] = "ref"    

    print(report_header)
    report = df[report_header].sort_values(by=['chromosome', 'pkt', 'start'])
    report.to_csv(filename_out, sep='\t', index=False, encoding='utf-8', header=True)
    return report
        

    
filename3 = "wuxi3"
xxx = generate_second_report(filename2, filename3, masked_genome)

['pkt', 'chromosome', 'start', 'stop', 'length', 'is_ref', 'left_K10B_num', 'left_K10B_rec', 'right_K10B_num', 'right_K10B_rec', 'left_K10f_num', 'left_K10f_rec', 'right_K10f_num', 'right_K10f_rec', 'left_LK5_1_3_num', 'left_LK5_1_3_rec', 'right_LK5_1_3_num', 'right_LK5_1_3_rec']


In [469]:
xxx.head()

,pkt,chromosome,start,stop,length,is_ref,left_K10B_num,left_K10B_rec,left_K10f_num,left_K10f_rec,left_LK5_1_3_num,left_LK5_1_3_rec
0,1,chromosome1,1895035,1895052,17,nonref,1,2c40afb0-7c90-4ddf-847c-a587e291d4a9,0,,6,"d7669e06-6916-435d-9925-6c207b9258ee,fe42f608-..."
1,1,chromosome1,18795334,18799060,3726,ref,1,ca50976c-e581-4f3d-a225-d6fb73c2b031,0,,5,"c81615f3-be5b-4c2b-9b60-bad3cfe7feb4,a597c928-..."
2,1,chromosome1,27115102,27115104,2,nonref,1,9ebca745-06aa-4187-9303-3fc73853bf74,0,,0,
3,1,chromosome1,40590596,40590593,-3,nonref,1,98c0c670-f831-449a-a2d7-6b50aad7d341,0,,0,
4,1,chromosome1,43955546,43955542,-4,nonref,1,bed145d3-72e9-4002-ba69-3a2a28356019,0,,0,


In [409]:
xxx

,pkt,chromosome,start,stop,length,is_ref,left_K10B_num,left_K10B_rec,left_K10f_num,left_K10f_rec,left_LK5_1_3_num,left_LK5_1_3_rec
0,1,chromosome1,1895035,1895057,22,nonref,0,,0,,3,"d7669e06-6916-435d-9925-6c207b9258ee,fe42f608-..."
1,1,chromosome1,18795334,18799060,3726,ref,0,,0,,5,"c81615f3-be5b-4c2b-9b60-bad3cfe7feb4,a597c928-..."
2,1,chromosome2,24991939,24991944,5,nonref,1,aa99dd0d-ba11-4f47-a79e-5dfedc9da5e2,0,,0,
3,1,chromosome2,30539627,30544358,4731,ref,1,b647706c-82df-4164-9168-4b200501ca6d,1,60b422a4-62dd-4740-bb99-d3de14200d9a,0,
4,1,chromosome3,27718081,27722607,4526,ref,1,6c07ed7c-c720-49ac-9e4b-b83309306bae,1,f374ba15-f870-4ad2-9daf-ff983b2a8970,0,
5,1,chromosome3,35158725,35158723,-2,nonref,1,cf60c0dc-a60a-4a67-acb7-28845f6eeb4c,0,,0,
6,1,chromosome3,41159456,41164183,4727,ref,3,"c89a6c1c-495b-4c98-bcb1-376e6b1b3fb4,7e4c3981-...",0,,0,
7,1,chromosome4,4759534,4759532,-2,nonref,1,941656fa-26ed-4578-8aab-59824b70f133,0,,0,
8,1,chromosome4,35867416,35867451,35,nonref,1,a7a007a4-c486-4df5-854d-69f74d61d3d1,0,,0,
9,2,chromosome4,26247770,26247794,24,nonref,0,,2,"b3e50c08-b937-486c-97ff-2ed57cab1f95,47eb892f-...",0,


In [117]:
x = 31
print(1, f"from {0/100*x} to {32/100*x}")
print(2, f"from {33/100*x} to {49/100*x}")
print(3, f"from {50/100*x} to {73/100*x}")
print(4, f"from {74/100*x} to {89/100*x}")
print(5, f"from {90/100*x} to {97/100*x}")
print(6, f"from {98/100*x} to {100/100*x}")

1 from 0.0 to 9.92
2 from 10.23 to 15.19
3 from 15.5 to 22.63
4 from 22.94 to 27.59
5 from 27.900000000000002 to 30.07
6 from 30.38 to 31.0
